In [ ]:
from typing import Tuple, List
from abc import ABC, abstractmethod
from decimal import Decimal

In [ ]:
import numpy as np
import pandas as pd
import math

In [ ]:
import os
import sys

root = os.path.split(os.getcwd())[0]
if root not in sys.path:
    sys.path.append(root)

In [ ]:
%load_ext autoreload
%autoreload 2
from strategy.Positions import AbstractPosition, UniV3Position, BiCurrencyPosition
from strategy.Portfolio import Portfolio
from strategy.Data import PoolDataUniV3
from strategy.Backtest import AbstractStrategy, Backtest

from strategy import PoolData, Pool, Token, Fee, Frequency, RawData

In [ ]:
def impermanent_loss(p_a, p_b, p_init, p):
    s_p_a, s_p_b = np.sqrt(p_a), np.sqrt(p_b)
    s_p_init, s_p = np.sqrt(p_init), np.sqrt(p)
    
    numer = 2 * s_p - s_p_a - p / s_p_b
    
    denom = s_p_init - s_p_a + (1 / s_p_init - 1 / s_p_b) * p
    
    return (numer / denom) - 1

In [ ]:
pool = Pool(Token.WBTC, Token.WETH, Fee.MIDDLE)

In [ ]:
data = PoolDataUniV3.from_folder(pool)
data.preprocess()

In [ ]:
data.plot()

In [ ]:
class MyStrategy(AbstractStrategy):
    def __init__(self,
                 uni_tolerance: int,
                 bicur_tolerance: int,
                 grid_width: int = 60,
                 width_num : int = 1,
                 decimal_diff: int = 10,
                 fees_percent = 0.003,
                 portfolio: Portfolio = None,
                ):
        super().__init__(portfolio)
        
        self.uni_tolerance = uni_tolerance
        self.bicur_tolerance = bicur_tolerance
        
        self.grid_width = grid_width
        self.width_num = width_num
        self.decimal_diff = decimal_diff
        self.fees_percent = fees_percent
        
        self.previous_uni_ticks = [258000]
        self.previous_equalize_price = 258000
        self.reinvest_prev_timestamp = pd.Timestamp('2021-05-04')
    
    
    def rebalance(self, *args, **kwargs) -> None:
        
        timestamp, row = kwargs['timestamp'], kwargs['row']
        price_0, price_1 = row['price_before'], row['price']
        
        tick_lower_bound, center_tick, tick_upper_bound = self._get_bounds_(price_1, self.decimal_diff, self.grid_width, self.width_num)
        current_tick = self._price_to_tick_(price_1, self.decimal_diff)
        
        for name, position in self.portfolio.positions.items():
            if hasattr(position, 'charge_fees'):
                position.charge_fees(price_0, price_1)
         
        nearest_pos = np.abs(np.asarray(self.previous_uni_ticks) - current_tick).min()
        if nearest_pos >= self.grid_width * (self.width_num + 1) - self.uni_tolerance:
            if abs(current_tick - center_tick) < self.uni_tolerance:
#                 print(f'''# UPDATE
#                             timestamp={timestamp},
#                             price={price_1}, 
#                             current_tick={current_tick},
#                             center={center_tick}, 
#                             lower_price={tick_lower_bound}, 
#                             upper_price={tick_upper_bound}, 
#                                 ''')
               
                fraction = 1.001**((tick_upper_bound - tick_lower_bound) / 2) - 1
                x_for_pos, y_for_pos = self.portfolio.get_position('Vault').withdraw_fraction(fraction)
                
                lower_price = self._tick_to_price_(tick_lower_bound, self.decimal_diff)
                upper_price = self._tick_to_price_(tick_upper_bound, self.decimal_diff)
                univ3_pos = UniV3Position(f'UniV3_{timestamp}', lower_price, upper_price, self.fees_percent)
        
                univ3_pos.deposit(x_for_pos, y_for_pos, price_1)
                self.portfolio.append(univ3_pos)
                
                self.previous_uni_ticks.append(center_tick)
            
        
        if abs(current_tick - self.previous_equalize_price) >= self.bicur_tolerance:
#             print(f'Prev_eq={self.previous_equalize_price}, Curr_eq={current_tick}')
            self.portfolio.get_position('Vault').equalize(price_1)
            self.previous_equalize_price = current_tick
            
        if self.reinvest_prev_timestamp < timestamp.normalize():
#             print(f'Prev_date={self.reinvest_prev_timestamp}, Cur_date={timestamp.normalize()}')
            
            for name, position in self.portfolio.positions.items():
                if hasattr(position, 'reinvest_fees'):
                    position.reinvest_fees(price_1)
                
            self.reinvest_prev_timestamp = timestamp.normalize()
        
        return None
    
        
    def snapshot(self, date, price: float) -> None:
        self.portfolio.snapshot(date, price)
        return None
    
    @staticmethod
    def _tick_to_price_(tick, decimal_diff):
        price = np.power(1.0001, tick) / 10**decimal_diff
        return price
    
    @staticmethod
    def _price_to_tick_(price, decimal_diff):
        tick = math.log(price, 1.0001) + decimal_diff * math.log(10, 1.0001)
        return int(round(tick))
    
    def _get_bounds_(self, price, decimal_diff, grid_width, width_num):
        current_tick =self._price_to_tick_(price, decimal_diff)
        center_num = int(current_tick // grid_width)
        center_tick = grid_width * center_num

        tick_lower_bound = grid_width * (center_num - width_num)
        tick_upper_bound = grid_width * (center_num + width_num)
        return tick_lower_bound, center_tick, tick_upper_bound

In [ ]:
c_price = 16

In [ ]:
bicurrency = BiCurrencyPosition('Vault', 0.003, 0.9 / c_price, 0.9)
univ3_pos_main = UniV3Position(f'UniV3_main', 16 / 1.0001**60, 16 * 1.0001**60, 0.003)
univ3_pos_main.deposit(0.1 / c_price, 0.1, c_price)

portfolio = Portfolio('main', [bicurrency, univ3_pos_main])

In [ ]:
portfolio.positions

In [ ]:
b = Backtest(MyStrategy(uni_tolerance=10, bicur_tolerance=240, portfolio=portfolio))

In [ ]:
b.backtest(data)

In [ ]:
# pd.DataFrame([portfolio.get_position('UniV3').history_y], index=['Volume_y']).T

In [ ]:
portfolio.draw_intervals(data)

In [ ]:
portfolio.draw_portfolio()

In [ ]:
stats = portfolio.portfolio_stats()

In [ ]:
stats

In [ ]:
swaps = data.swaps
mints = data.mints
burns = data.burns

In [ ]:
swaps.loc[swaps.index > pd.Timestamp('2021-09-16')]

In [ ]:
swaps.sort_index()[-1605:]